<a href="https://www.kaggle.com/code/sulaniishara/sticker-sales-analysis-forecasting?scriptVersionId=221258267" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<!-- Include Google Fonts for a modern font -->
<link href="https://fonts.googleapis.com/css2?family=Roboto:wght@700&display=swap" rel="stylesheet">

<div style="border-radius: 15px; border: 2px solid #6A1B9A; padding: 20px; 
           background: linear-gradient(135deg, #6610f2, #ff99cc, #ffa07a, #ffff00); 
           text-align: center; box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.5);">
    <h1 style="color: #ffffff; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.7); 
               font-weight: bold; margin-bottom: 10px; font-size: 36px; 
               font-family: 'Roboto', sans-serif; line-height: 1.2;">
        📅 Sticker Sales Analysis & Forecasting 📈
    </h1>
</div>


#### **📊 Data Description**  
The dataset for this project consists of historical sales data for various products sold by a retailer. The data includes the following key components:

1. **📂 Training Data (`train.csv`)**  
   - Contains historical sales records used for training the model. Key columns include:  
     - `id`: 🆔 Unique identifier for each record.  
     - `date`: 📅 The date when sales were recorded.  
     - `country`: 🌍 The country where the sales occurred (e.g., Canada, Finland, Italy).  
     - `store`: 🏬 The store type (e.g., Discount Stickers, Stickers for Less).  
     - `product`: 🛍️ The product type (e.g., Holographic Goose, Kaggle).  
     - `num_sold`: 🔢 The number of units sold (target variable).  

2. **📂 Test Data (`test.csv`)**  
   - Contains future sales records without the `num_sold` column. The objective is to predict the values for this column.  

3. **📄 Sample Submission File (`sample_submission.csv`)**  
   - Provides the expected format for the submission, with `id` and `num_sold` columns. The predictions will replace the `num_sold` column.  

4. **📊 Supplementary Data**  
   - Ratios and weights such as day-of-week ratios, GDP per capita adjustments, and store weights are used to model and disaggregate sales trends.  

#### **🎯 Objective**  
The primary objective of this project is to **forecast the number of products sold (`num_sold`) for each record in the test dataset**. This involves:  

1. **🔍 Understanding Sales Patterns**  
   Analyzing historical sales data to identify patterns influenced by:  
   - 📆 Seasonality (e.g., holidays, weekends).  
   - 🏢 Store types and country-specific trends.  
   - 📈 Product demand variations.  

2. **🛠️ Building a Robust Forecasting Model**  
   Using techniques like:  
   - 🔄 Imputation for handling missing data in historical records.  
   - 📊 Aggregation and disaggregation to model total and individual sales accurately.  
   - ⚖️ Ratio-based adjustments for finer granularity (e.g., GDP per capita, day-of-week effects).  

3. **💡 Providing Actionable Insights**  
   The final model will:  
   - 📦 Help optimize inventory management and resource allocation across stores and regions.  
   - 📊 Provide actionable insights into market demand trends for better strategic planning.  

Through the accurate prediction of future sales, this project aims to support **data-driven decision-making** 🧠 and enhance **profitability** 💰.

## Importing Required Libraries


In [ ]:
# Importing necessary libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from scipy.signal import find_peaks
import requests
import plotly.express as px
import plotly.io as pio

import warnings
warnings.filterwarnings("ignore")

## Loading and Verifying the Datasets

In [ ]:
# Load the datasets
train_df = pd.read_csv("/kaggle/input/playground-series-s5e1/train.csv", parse_dates=["date"])
test_df = pd.read_csv("/kaggle/input/playground-series-s5e1/test.csv", parse_dates=["date"])
# sample_df = pd.read_csv("/kaggle/input/playground-series-s5e1/sample_submission.csv")

# Verify shapes
print("Train Data Shape:", train_df.shape)
print("Test Data Shape:", test_df.shape)

## Displaying Sample Data

In [ ]:
# Display sample data
print("Training Dataset: \n")
display(train_df.head(10))
print('\n')
print("Test Dataset: \n")
display(test_df.head(10))

## Dataset Information and Summary

In [ ]:
# Display information for the training dataset
print("Training Dataset Information: \n")
train_info = train_df.info()
display(train_info)
print('\n')
# Display information for the test dataset
print("Test Dataset Information: \n")
test_info = test_df.info()
display(test_info)

## Analyzing Unique Values in Columns

In [ ]:
# Display the number of unique values in each column for train_df
print("\nUnique Values in Each Column (Train Data):")
print(train_df.nunique())

# Display the lists of numerical and categorical columns in train_df
non_numerical_columns_train = train_df.select_dtypes(include=['object']).columns.tolist()
print("\nCategorical Columns (Train Data):", non_numerical_columns_train)

# Display unique values for each categorical column in train_df
for col in non_numerical_columns_train:
    print(f"\nColumn: {col}")
    print(f"Unique Values: {train_df[col].unique()}")

# Display the number of unique values in each column for test_df
print("\nUnique Values in Each Column (Test Data):")
print(test_df.nunique())

# Display the lists of numerical and categorical columns in test_df
non_numerical_columns_test = test_df.select_dtypes(include=['object']).columns.tolist()
print("\nCategorical Columns (Test Data):", non_numerical_columns_test)

# Display unique values for each categorical column in test_df
for col in non_numerical_columns_test:
    print(f"\nColumn: {col}")
    print(f"Unique Values: {test_df[col].unique()}")

#### **Insights from Data Exploration**

#### Dataset Overview

- **Training Data**: The training dataset comprises **230,130 entries** and **6 columns**: `id`, `date`, `country`, `store`, `product`, and `num_sold`. Notably, the `num_sold` column has missing values, with only **221,259 non-null entries**.
- **Test Data**: The test dataset consists of **98,550 entries** and **5 columns**: `id`, `date`, `country`, `store`, and `product`.

#### Unique Values and Categorical Columns

- **Unique Values**:
  - Both datasets share the same unique values for the columns `country`, `store`, and `product`.
    - **`country`**: 6 unique values — *Canada, Finland, Italy, Kenya, Norway, Singapore*.
    - **`store`**: 3 unique values — *Discount Stickers, Stickers for Less, Premium Sticker Mart*.
    - **`product`**: 5 unique values — *Holographic Goose, Kaggle, Kaggle Tiers, Kerneler, Kerneler Dark Mode*.

- **Categorical Columns**:
  - The categorical columns in both datasets are **`country`**, **`store`**, and **`product`**.

#### Missing Values

- **Training Data**: The `num_sold` column has missing values, with approximately **8,871 entries** lacking data.


## Fetching GDP Per Capita for a Country and Year

In [ ]:
# Function to fetch GDP per capita for a given country and year
def get_gdp_per_capita(alpha3, year):
    """
    Fetch GDP per capita for a specific country and year from the World Bank API.
    
    """
    url = f'https://api.worldbank.org/v2/country/{alpha3}/indicator/NY.GDP.PCAP.CD?date={year}&format=json'
    try:
        response = requests.get(url)
        response.raise_for_status()  
        data = response.json()
        return data[1][0]['value'] if data[1] else None  
    except (requests.RequestException, KeyError, IndexError) as e:
        print(f"Error fetching data for {alpha3} in {year}: {e}")
        return None


## Normalizing GDP Ratios

In [ ]:
# Function to create a DataFrame of GDP ratios
def create_gdp_dataframe(alpha3s, years, country_names):
    """
    Create a DataFrame of normalized GDP per capita ratios for multiple countries and years.
    
    """
    # Fetch GDP data for all countries and years
    gdp_data = [
        [get_gdp_per_capita(alpha3, year) for year in years]
        for alpha3 in alpha3s
    ]
    
    # Create a DataFrame with countries as rows and years as columns
    gdp_df = pd.DataFrame(gdp_data, index=country_names, columns=years)
    
    # Normalize GDP values by dividing by the column sum (yearly total)
    gdp_df = gdp_df / gdp_df.sum(axis=0)
    
    # Reshape the DataFrame into long format
    gdp_df = gdp_df.reset_index().rename(columns={'index': 'country'})
    gdp_df = gdp_df.melt(id_vars=['country'], var_name='year', value_name='ratio')
    
    return gdp_df


## Adjusting GDP Ratios for Specific Countries

In [ ]:
# Function to adjust ratios for specific countries
def adjust_ratios(gdp_df, adjustments):
    """
    Adjust GDP ratios for specific countries based on custom rules.
    
    """
    adjusted_df = gdp_df.copy()
    for country, adjustment in adjustments.items():
        adjusted_df.loc[adjusted_df['country'] == country, 'ratio'] -= adjustment
    return adjusted_df

if __name__ == "__main__":
    # Define input parameters
    alpha3s = ['CAN', 'FIN', 'ITA', 'KEN', 'NOR', 'SGP']
    years = range(2010, 2020)
    country_names = np.sort(['Canada', 'Finland', 'Italy', 'Kenya', 'Norway', 'Singapore'])  # Example list
    
    # Create the GDP DataFrame
    gdp_ratios_df = create_gdp_dataframe(alpha3s, years, country_names)
    
    # Adjust Kenya's ratio by subtracting 0.0007
    adjustments = {'Kenya': 0.0007}
    gdp_per_capita_filtered_ratios_df = adjust_ratios(gdp_ratios_df, adjustments)
    
    print(gdp_per_capita_filtered_ratios_df.head(6))


## Visualizing GDP Ratios with a Choropleth Map

In [ ]:
pio.renderers.default = 'iframe'

# Filter for a specific year, e.g., 2010
gdp_2010 = gdp_per_capita_filtered_ratios_df[gdp_per_capita_filtered_ratios_df['year'] == 2010]

# Plot choropleth map
fig = px.choropleth(
    gdp_2010,
    locations='country',
    locationmode='country names',
    color='ratio',
    hover_name='country',
    title='Normalized GDP Per Capita Ratios (2010)',
    color_continuous_scale=px.colors.sequential.Plasma
)
fig.show()


## Heatmap of Missing Values in the Training Dataset

In [ ]:
# Visualize missing values with a heatmap
plt.figure(figsize=(10, 4))
sns.heatmap(train_df.isnull(), cbar=False, cmap="plasma")
plt.title("Heatmap of Missing Values in Training Dataset")
plt.show()

* The heatmap highlights missing values in the training dataset, primarily in the num_sold column. This indicates that some rows lack the target variable, which must be handled carefully during preprocessing.

## Observing Missing Values in the num_sold Column

In [ ]:
# Count of missing and non-missing values
missing_count = train_df['num_sold'].isnull().sum()
non_missing_count = len(train_df) - missing_count

# Define colors using the Plasma color palette
colors = [plt.cm.plasma(0.1), plt.cm.plasma(0.7)]

# Create subplots: one for the bar chart and one for the donut chart
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.bar(['Non-missing', 'Missing'], [non_missing_count, missing_count], color=colors)
ax1.set_title('Distribution of Missing Values in num_sold')
ax1.set_ylabel('Count')
ax1.grid(True, linestyle='--', alpha=0.6)

for i, count in enumerate([non_missing_count, missing_count]):
    ax1.text(i, count + 0.5, str(count), ha='center', va='bottom')

labels = ['Non-missing', 'Missing']
sizes = [non_missing_count, missing_count]
ax2.pie(sizes, labels=labels, autopct='%1.1f%%', colors=colors, startangle=90,
        wedgeprops=dict(width=0.3))  
ax2.axis('equal')  
ax2.set_title('Proportion of Missing Values in num_sold')

plt.tight_layout()
plt.show()


#### Observational Insights

1. **Missing vs. Non-Missing Counts**:
   - **Non-Missing Values**: 221,259 entries (96.1% of the dataset).
   - **Missing Values**: 8,871 entries (3.9% of the dataset).

The majority of the `num_sold` column is populated with valid data, with only a small fraction of rows containing missing values.


## Temporal Analysis of Missing Values Over Time

In [ ]:
# Create a new column to indicate missingness
train_df['num_sold_missing'] = train_df['num_sold'].isnull()

# Group by date and calculate the percentage of missing values
missing_by_date = train_df.groupby('date')['num_sold_missing'].mean()

plt.figure(figsize=(12, 4))
missing_by_date.plot(kind='line', color=plt.cm.plasma(0.2), alpha=0.7)
plt.title("Trend of Missing 'num_sold' Values Over Time")
plt.xlabel('Date')
plt.ylabel('Missing Value Percentage')
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

print(missing_by_date.describe())

* The time-series plot shows relatively consistent missingness, with occasional peaks, indicating certain dates may have experienced more missing entries.

## Analysis of Missing Values by Country

In [ ]:
# Count missing values by country
missing_by_country = train_df[train_df['num_sold_missing']]['country'].value_counts()

plt.figure(figsize=(12, 4))
missing_by_country.plot(kind='bar', color=plt.cm.plasma(0.8))
plt.title("Missing 'num_sold' Values by Country")
plt.xlabel("Country")
plt.ylabel("Count of Missing Values")
plt.xticks(rotation=45)
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

print(missing_by_country.describe())


The plot shows the count of missing values for the "num_sold" column across two countries: Kenya and Canada.
Kenya has a significantly higher number of missing values compared to Canada.

**Specific Insights:**

* **Kenya:** This country has the highest number of missing values for "num_sold".
* **Canada:** This country has a lower number of missing values compared to Kenya.

## Analysis of Missing Values by Store

In [ ]:
# Count missing values by store
missing_by_store = train_df[train_df['num_sold_missing']]['store'].value_counts()

plt.figure(figsize=(12, 4))
missing_by_store.plot(kind='bar', color=plt.cm.plasma(0.4))
plt.title("Missing 'num_sold' Values by Store")
plt.xlabel("Store")
plt.ylabel("Count of Missing Values")
plt.xticks(rotation=45)
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

print(missing_by_store.describe())


The plot shows the count of missing values for the "num_sold" column across three different stores: Discount Stickers, Stickers for Less, and Premium Sticker Mart.
There is a significant variation in the number of missing values across the stores.

**Specific Insights:**

* **Discount Stickers:** This store has the highest number of missing values for "num_sold".
* **Stickers for Less:** This store has the second-highest number of missing values.
* **Premium Sticker Mart:** This store has the lowest number of missing values.

## Analysis of Missing Values by Product

In [ ]:
# Count missing values by product
missing_by_product = train_df[train_df['num_sold_missing']]['product'].value_counts()

plt.figure(figsize=(12, 4))
missing_by_product.plot(kind='bar', color=plt.cm.plasma(0.5))
plt.title("Missing 'num_sold' Values by Product")
plt.xlabel("Product")
plt.ylabel("Count of Missing Values")
plt.xticks(rotation=45)
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

print(missing_by_product.describe())


The plot shows the count of missing values for the "num_sold" column across three different products: Holographic Goose, Kerneler, and Kerneler Dark Mode.
There is a significant variation in the number of missing values across the products.

**Specific Insights:**

* **Holographic Goose:** This product has the highest number of missing values for "num_sold".
* **Kerneler:** This product has a much lower number of missing values compared to Holographic Goose.
* **Kerneler Dark Mode:** This product has the least number of missing values, with only a very small number of missing values.

## Analyzing Missing Values by Country and Store

In [ ]:
# Count missing values grouped by both country and store
missing_by_country_store = (
    train_df[train_df['num_sold_missing']]
    .groupby(['country', 'store'])['num_sold']
    .size()
    .reset_index(name='missing_count')
)

plt.figure(figsize=(14, 6))

# Create a bar plot for missing values by store within each country
sns.barplot(
    data=missing_by_country_store,
    x='store',
    y='missing_count',
    hue='country',
    palette='plasma'
)

plt.title("Missing 'num_sold' Values by Store for Each Country", fontsize=16)
plt.xlabel("Store", fontsize=12)
plt.ylabel("Count of Missing Values", fontsize=12)
plt.xticks(rotation=45, fontsize=12)
plt.legend(title='Country', fontsize=12, title_fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

The plot visualizes the count of missing values for the "num_sold" column across three different stores (Discount Stickers, Premium Sticker Mart, Stickers for Less) and two countries (Canada and Kenya).
There is a noticeable variation in the number of missing values across stores and between the two countries.

**Specific Insights:**

* **Discount Stickers:** This store has the highest number of missing values for "num_sold" in both Canada and Kenya.
* **Stickers for Less:** This store has a moderate number of missing values in both countries.
* **Premium Sticker Mart:** This store has the lowest number of missing values in both countries.
* **Country Comparison:**
    * In general, Discount Stickers has a significantly higher number of missing values in Kenya compared to Canada.
    * For Stickers for Less and Premium Sticker Mart, the difference in missing values between Canada and Kenya is less pronounced.

## Analyzing Missing Values by Country and Product

In [ ]:
# Count missing values grouped by both country and product
missing_by_country_product = (
    train_df[train_df['num_sold_missing']]
    .groupby(['country', 'product'])['num_sold']
    .size()
    .reset_index(name='missing_count')
)

plt.figure(figsize=(14, 6))

# Create a bar plot for missing values by product within each country
sns.barplot(
    data=missing_by_country_product,
    x='product',
    y='missing_count',
    hue='country',
    palette='plasma'
)

plt.title("Missing 'num_sold' Values by Product for Each Country", fontsize=16)
plt.xlabel("Product", fontsize=12)
plt.ylabel("Count of Missing Values", fontsize=12)
plt.xticks(rotation=45, fontsize=12)
plt.legend(title='Country', fontsize=12, title_fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


The plot visualizes the count of missing values for the "num_sold" column across three different products (Holographic Goose, Kerneler, Kerneler Dark Mode) and two countries (Canada and Kenya).
There is a significant variation in the number of missing values across products and between the two countries.

**Specific Insights:**

* **Holographic Goose:** This product has the highest number of missing values in both Canada and Kenya.
* **Kerneler:** This product has a much lower number of missing values compared to Holographic Goose in both countries.
* **Kerneler Dark Mode:** This product has the least number of missing values, with only a very small number of missing values in both countries.
* **Country Comparison:**
    * For Holographic Goose, the number of missing values is higher in Kenya compared to Canada.
    * For Kerneler and Kerneler Dark Mode, the difference in missing values between Canada and Kenya is less pronounced.

## Analyzing Missing Values by Store and Product

In [ ]:
# Count missing values grouped by both store and product
missing_by_store_product = (
    train_df[train_df['num_sold_missing']]
    .groupby(['store', 'product'])['num_sold']
    .size()
    .reset_index(name='missing_count')
)

plt.figure(figsize=(14, 6))

# Create a bar plot for missing values by product within each store
sns.barplot(
    data=missing_by_store_product,
    x='product',
    y='missing_count',
    hue='store',
    palette='plasma'
)

plt.title("Missing 'num_sold' Values by Product for Each Store", fontsize=16)
plt.xlabel("Product", fontsize=12)
plt.ylabel("Count of Missing Values", fontsize=12)
plt.xticks(rotation=45, fontsize=12)
plt.legend(title='Store', fontsize=12, title_fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


The plot visualizes the count of missing values for the "num_sold" column across three different products (Holographic Goose, Kerneler, Kerneler Dark Mode) and three different stores (Discount Stickers, Premium Sticker Mart, Stickers for Less).
There is a significant variation in the number of missing values across products and stores.

**Specific Insights:**

* **Holographic Goose:** This product has the highest number of missing values across all stores. 
    * **Discount Stickers:** This store has the highest number of missing values for Holographic Goose.
    * **Premium Sticker Mart:** This store has a moderate number of missing values for Holographic Goose.
    * **Stickers for Less:** This store has a relatively low number of missing values for Holographic Goose.

* **Kerneler:** This product has a much lower number of missing values compared to Holographic Goose across all stores.
    * **Discount Stickers:** This store has the highest number of missing values for Kerneler.
    * **Premium Sticker Mart:** This store has a very low number of missing values for Kerneler.
    * **Stickers for Less:** This store has a very low number of missing values for Kerneler.

* **Kerneler Dark Mode:** This product has the least number of missing values across all stores. 
    * **Discount Stickers:** This store has the highest number of missing values for Kerneler Dark Mode.
    * **Premium Sticker Mart:** This store has a very low number of missing values for Kerneler Dark Mode.
    * **Stickers for Less:** This store has a very low number of missing values for Kerneler Dark Mode.

### Dropping the Temporary Missingness Column

In [ ]:
# Dropping specified column from train_data
train_df = train_df.drop('num_sold_missing', axis=1)

# Display the updated DataFrame to confirm the columns have been dropped
print("Updated train_data after dropping specified columns:")
print(train_df.head())


## Imputation of Missing Sales Data

In [ ]:
# Create a copy of the DataFrame
train_df_imputed = train_df.copy()
print(f"Missing values remaining: {train_df_imputed['num_sold'].isna().sum()}")

# Extract the year from the date
train_df_imputed["year"] = train_df_imputed["date"].dt.year

# Loop through each year to perform imputation
for year in train_df_imputed["year"].unique():
    # Target ratio (Norway)
    target_ratio = gdp_per_capita_filtered_ratios_df.loc[
        (gdp_per_capita_filtered_ratios_df["year"] == year) & 
        (gdp_per_capita_filtered_ratios_df["country"] == "Norway"), "ratio"
    ].values[0]

    # Impute Time Series 1: Canada, Discount Stickers, Holographic Goose
    current_ratio_can = gdp_per_capita_filtered_ratios_df.loc[
        (gdp_per_capita_filtered_ratios_df["year"] == year) & 
        (gdp_per_capita_filtered_ratios_df["country"] == "Canada"), "ratio"
    ].values[0]
    ratio_can = current_ratio_can / target_ratio
    train_df_imputed.loc[
        (train_df_imputed["country"] == "Canada") & 
        (train_df_imputed["store"] == "Discount Stickers") & 
        (train_df_imputed["product"] == "Holographic Goose") & 
        (train_df_imputed["year"] == year), 
        "num_sold"
    ] = (
        train_df_imputed.loc[
            (train_df_imputed["country"] == "Norway") & 
            (train_df_imputed["store"] == "Discount Stickers") & 
            (train_df_imputed["product"] == "Holographic Goose") & 
            (train_df_imputed["year"] == year), 
            "num_sold"
        ] * ratio_can
    ).values

    # Impute Time Series 2-3: Canada, Premium Sticker Mart / Stickers for Less
    for store in ["Premium Sticker Mart", "Stickers for Less"]:
        current_ts = train_df_imputed.loc[
            (train_df_imputed["country"] == "Canada") & 
            (train_df_imputed["store"] == store) & 
            (train_df_imputed["product"] == "Holographic Goose") & 
            (train_df_imputed["year"] == year)
        ]
        missing_ts_dates = current_ts.loc[current_ts["num_sold"].isna(), "date"]
        train_df_imputed.loc[
            (train_df_imputed["country"] == "Canada") & 
            (train_df_imputed["store"] == store) & 
            (train_df_imputed["product"] == "Holographic Goose") & 
            (train_df_imputed["year"] == year) & 
            (train_df_imputed["date"].isin(missing_ts_dates)), 
            "num_sold"
        ] = (
            train_df_imputed.loc[
                (train_df_imputed["country"] == "Norway") & 
                (train_df_imputed["store"] == store) & 
                (train_df_imputed["product"] == "Holographic Goose") & 
                (train_df_imputed["year"] == year) & 
                (train_df_imputed["date"].isin(missing_ts_dates)), 
                "num_sold"
            ] * ratio_can
        ).values

    # Impute Time Series 4: Kenya, Discount Stickers, Holographic Goose
    current_ratio_ken = gdp_per_capita_filtered_ratios_df.loc[
        (gdp_per_capita_filtered_ratios_df["year"] == year) & 
        (gdp_per_capita_filtered_ratios_df["country"] == "Kenya"), "ratio"
    ].values[0]
    ratio_ken = current_ratio_ken / target_ratio
    train_df_imputed.loc[
        (train_df_imputed["country"] == "Kenya") & 
        (train_df_imputed["store"] == "Discount Stickers") & 
        (train_df_imputed["product"] == "Holographic Goose") & 
        (train_df_imputed["year"] == year), 
        "num_sold"
    ] = (
        train_df_imputed.loc[
            (train_df_imputed["country"] == "Norway") & 
            (train_df_imputed["store"] == "Discount Stickers") & 
            (train_df_imputed["product"] == "Holographic Goose") & 
            (train_df_imputed["year"] == year), 
            "num_sold"
        ] * ratio_ken
    ).values

    # Impute Time Series 5-6: Kenya, Premium Sticker Mart / Stickers for Less
    for store in ["Premium Sticker Mart", "Stickers for Less"]:
        current_ts = train_df_imputed.loc[
            (train_df_imputed["country"] == "Kenya") & 
            (train_df_imputed["store"] == store) & 
            (train_df_imputed["product"] == "Holographic Goose") & 
            (train_df_imputed["year"] == year)
        ]
        missing_ts_dates = current_ts.loc[current_ts["num_sold"].isna(), "date"]
        train_df_imputed.loc[
            (train_df_imputed["country"] == "Kenya") & 
            (train_df_imputed["store"] == store) & 
            (train_df_imputed["product"] == "Holographic Goose") & 
            (train_df_imputed["year"] == year) & 
            (train_df_imputed["date"].isin(missing_ts_dates)), 
            "num_sold"
        ] = (
            train_df_imputed.loc[
                (train_df_imputed["country"] == "Norway") & 
                (train_df_imputed["store"] == store) & 
                (train_df_imputed["product"] == "Holographic Goose") & 
                (train_df_imputed["year"] == year) & 
                (train_df_imputed["date"].isin(missing_ts_dates)), 
                "num_sold"
            ] * ratio_ken
        ).values

    # Impute Time Series 7: Kenya, Discount Stickers, Kerneler
    current_ts = train_df_imputed.loc[
        (train_df_imputed["country"] == "Kenya") & 
        (train_df_imputed["store"] == "Discount Stickers") & 
        (train_df_imputed["product"] == "Kerneler") & 
        (train_df_imputed["year"] == year)
    ]
    missing_ts_dates = current_ts.loc[current_ts["num_sold"].isna(), "date"]
    train_df_imputed.loc[
        (train_df_imputed["country"] == "Kenya") & 
        (train_df_imputed["store"] == "Discount Stickers") & 
        (train_df_imputed["product"] == "Kerneler") & 
        (train_df_imputed["year"] == year) & 
        (train_df_imputed["date"].isin(missing_ts_dates)), 
        "num_sold"
    ] = (
        train_df_imputed.loc[
            (train_df_imputed["country"] == "Norway") & 
            (train_df_imputed["store"] == "Discount Stickers") & 
            (train_df_imputed["product"] == "Kerneler") & 
            (train_df_imputed["year"] == year) & 
            (train_df_imputed["date"].isin(missing_ts_dates)), 
            "num_sold"
        ] * ratio_ken
    ).values

# Check for remaining missing values
print(f"Missing values remaining after imputation: {train_df_imputed['num_sold'].isna().sum()}")

# Manual imputation for specific IDs
train_df_imputed.loc[train_df_imputed["id"] == 23719, "num_sold"] = 4
train_df_imputed.loc[train_df_imputed["id"] == 207003, "num_sold"] = 195

# Final check for missing values
print(f"Final missing values remaining: {train_df_imputed['num_sold'].isna().sum()}")


## Detection of Duplicates

In [ ]:
# Check for duplicate rows in the training dataset
train_duplicates = train_df_imputed.duplicated().sum()
print(f"\nNumber of duplicate rows in the training dataset: {train_duplicates}")

# Check for duplicate rows in the test dataset
test_duplicates = test_df.duplicated().sum()
print(f"Number of duplicate rows in the test dataset: {test_duplicates}")


## Distribution of num_sold

In [ ]:
# Custom colormap using Plasma
plasma_cmap = cm.get_cmap("plasma")

def visualize_num_sold_with_peaks(data, feature='num_sold'):
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    ax = sns.histplot(data[feature], bins=30, kde=True, color=plasma_cmap(0.5))
    plt.title(f'Histogram of {feature} with KDE', fontsize=12)
    plt.xlabel(feature, fontsize=10)
    plt.ylabel('Frequency', fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.6)

    kde = sns.kdeplot(data[feature], ax=ax, color=plasma_cmap(0.7)).lines[0].get_data()
    kde_x, kde_y = kde[0], kde[1]
    peaks, _ = find_peaks(kde_y)

    for peak_idx in peaks:
        plt.plot(kde_x[peak_idx], kde_y[peak_idx], "ro")  

    plt.subplot(1, 2, 2)
    sns.boxplot(x=data[feature], color=plasma_cmap(0.5))
    plt.title(f'Box Plot of {feature}', fontsize=12)
    plt.xlabel(feature, fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

visualize_num_sold_with_peaks(train_df_imputed, feature='num_sold')

## Distribution of country, store, and product

In [ ]:
# Function to display bar plot and pie chart for categorical columns
def plot_categorical_distribution(data, column_name):
    plasma_colors = sns.color_palette("plasma", data[column_name].nunique())
    
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    sns.countplot(y=column_name, data=data, palette=plasma_colors)
    plt.title(f'Distribution of {column_name}', fontsize=12)
    plt.xlabel('Count', fontsize=10)
    plt.ylabel(column_name, fontsize=10)

    ax = plt.gca()
    for p in ax.patches:
        count = int(p.get_width())
        ax.annotate(f'{count}', 
                    (p.get_width() + 0.1, p.get_y() + p.get_height() / 2), 
                    ha='left', va='center', fontsize=10, color='black')
    
    sns.despine(left=True, bottom=True)
    
    # Pie chart for percentage distribution
    plt.subplot(1, 2, 2)
    data[column_name].value_counts().plot.pie(
        autopct='%1.1f%%', 
        colors=plasma_colors, 
        startangle=90, 
        explode=[0.05] * data[column_name].nunique(), 
        shadow=True
    )
    plt.title(f'Percentage Distribution of {column_name}', fontsize=12)
    plt.ylabel('')  

    plt.tight_layout()
    plt.show()

categorical_columns = ['country', 'store', 'product']
for column in categorical_columns:
    plot_categorical_distribution(train_df_imputed, column)

## Distribution of num_sold across categorical features

In [ ]:
def facetgrid_and_boxplot(data, categorical_column, target_column):
    g = sns.FacetGrid(data, col=categorical_column, col_wrap=3, height=4, sharex=False, sharey=False, palette="plasma")
    g.map(sns.histplot, target_column, kde=False, bins=30, color=sns.color_palette("plasma")[0])
    g.set_titles("{col_name}")
    g.set_axis_labels(target_column, "Frequency")
    g.fig.suptitle(f"Distribution of {target_column} across unique values of {categorical_column}", y=1.02, fontsize=16)
    g.tight_layout()
    plt.show()

    plt.figure(figsize=(10, 4))
    sns.boxplot(x=categorical_column, y=target_column, data=data, palette="plasma")
    plt.title(f"Boxplot of {target_column} by {categorical_column}", fontsize=12)
    plt.xlabel(categorical_column, fontsize=10)
    plt.ylabel(target_column, fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

target_column = "num_sold"
categorical_column = "country"

facetgrid_and_boxplot(train_df_imputed, categorical_column, target_column)

In [ ]:
def facetgrid_and_boxplot(data, categorical_column, target_column):
    g = sns.FacetGrid(data, col=categorical_column, col_wrap=3, height=4, sharex=False, sharey=False, palette="plasma")
    g.map(sns.histplot, target_column, kde=False, bins=30, color=sns.color_palette("plasma")[1])
    g.set_titles("{col_name}")
    g.set_axis_labels(target_column, "Frequency")
    g.fig.suptitle(f"Distribution of {target_column} across unique values of {categorical_column}", y=1.02, fontsize=16)
    g.tight_layout()
    plt.show()

    plt.figure(figsize=(10, 4))
    sns.boxplot(x=categorical_column, y=target_column, data=data, palette="plasma")
    plt.title(f"Boxplot of {target_column} by {categorical_column}", fontsize=12)
    plt.xlabel(categorical_column, fontsize=10)
    plt.ylabel(target_column, fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

target_column = "num_sold"
categorical_column = "store"

facetgrid_and_boxplot(train_df_imputed, categorical_column, target_column)

In [ ]:
def facetgrid_and_boxplot(data, categorical_column, target_column):
    g = sns.FacetGrid(data, col=categorical_column, col_wrap=3, height=4, sharex=False, sharey=False, palette="plasma")
    g.map(sns.histplot, target_column, kde=False, bins=30, color=sns.color_palette("plasma")[2])
    g.set_titles("{col_name}")
    g.set_axis_labels(target_column, "Frequency")
    g.fig.suptitle(f"Distribution of {target_column} across unique values of {categorical_column}", y=1.02, fontsize=16)
    g.tight_layout()
    plt.show()

    plt.figure(figsize=(10, 4))
    sns.boxplot(x=categorical_column, y=target_column, data=data, palette="plasma")
    plt.title(f"Boxplot of {target_column} by {categorical_column}", fontsize=12)
    plt.xlabel(categorical_column, fontsize=10)
    plt.ylabel(target_column, fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

target_column = "num_sold"
categorical_column = "product"

facetgrid_and_boxplot(train_df_imputed, categorical_column, target_column)

## Daily Sales Trends
Aggregating sales by date and visualizing daily sales trends

In [ ]:
# Aggregate sales by date
daily_sales = train_df_imputed.groupby('date')['num_sold'].sum().reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(data=daily_sales, x='date', y='num_sold', color=sns.color_palette("plasma")[0])
plt.title('Daily Sales Trend', fontsize=14)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Total Sales', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

## Daily Sales Trends by Categories
Visualizing daily sales trends for each category (country, store, product) 

In [ ]:
categorical_columns = ['country', 'store', 'product']

# Set up the subplots for daily sales trends
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(24, 8), sharey=True)

# Loop through each category and create a subplot for daily trends
for i, category in enumerate(categorical_columns):
    # Aggregate sales by date and category
    category_sales_daily = train_df_imputed.groupby(['date', category])['num_sold'].sum().reset_index()
    
    sns.lineplot(
        data=category_sales_daily,
        x='date',
        y='num_sold',
        hue=category,
        palette='plasma',
        linewidth=1,
        ax=axes[i]
    )
    
    axes[i].set_title(f'Daily Sales Trend by {category.capitalize()}', fontsize=14)
    axes[i].set_xlabel('Date', fontsize=12)
    axes[i].set_ylabel('Total Sales', fontsize=12 if i == 0 else 0)  
    axes[i].legend(title=category.capitalize(), fontsize=10)
    axes[i].grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

## Contribution to Total Sales by Categories
Calculating and visualizing weights (proportional contributions) for country, store, and product.

In [ ]:
# Function to calculate weights of each country based on total sales
def calculate_country_weights(df):
    """
    Calculate the weights of each country based on total sales.
    """
    total_sales = df["num_sold"].sum()
    country_weights = df.groupby("country")["num_sold"].sum() / total_sales
    return country_weights

# Function to calculate weights of each store based on total sales
def calculate_store_weights(df):
    """
    Calculate the weights of each store based on total sales.
    """
    total_sales = df["num_sold"].sum()
    store_weights = df.groupby("store")["num_sold"].sum() / total_sales
    return store_weights

# Function to calculate weights of each product based on total sales
def calculate_product_weights(df):
    """
    Calculate the weights of each product based on total sales.
    """
    total_sales = df["num_sold"].sum()
    product_weights = df.groupby("product")["num_sold"].sum() / total_sales
    return product_weights

# Calculate weights
country_weights = calculate_country_weights(train_df_imputed)
store_weights = calculate_store_weights(train_df_imputed)
product_weights = calculate_product_weights(train_df_imputed)

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Plot Country Weights
sns.barplot(x=country_weights.index, y=country_weights.values, palette='plasma', ax=axes[0])
axes[0].set_title('Country Contribution to Total Sales', fontsize=12)
axes[0].set_xlabel('Country', fontsize=12)
axes[0].set_ylabel('Weight (Proportion of Total Sales)', fontsize=12)
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(axis='y', linestyle='--', alpha=0.6)

# Plot Store Weights
sns.barplot(x=store_weights.index, y=store_weights.values, palette='plasma', ax=axes[1])
axes[1].set_title('Store Contribution to Total Sales', fontsize=12)
axes[1].set_xlabel('Store', fontsize=12)
axes[1].set_ylabel('Weight (Proportion of Total Sales)', fontsize=12)
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(axis='y', linestyle='--', alpha=0.6)

# Plot Product Weights
sns.barplot(x=product_weights.index, y=product_weights.values, palette='plasma', ax=axes[2])
axes[2].set_title('Product Contribution to Total Sales', fontsize=12)
axes[2].set_xlabel('Product', fontsize=12)
axes[2].set_ylabel('Weight (Proportion of Total Sales)', fontsize=12)
axes[2].tick_params(axis='x', rotation=45)
axes[2].grid(axis='y', linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()


In [ ]:
def decompose(data, group_col, ax, colormap='plasma'):
    """
    Decomposes and plots the fraction of sales for each group (e.g., store, product) over time
    
    """
    # Group data by date and group_col, then calculate total sales
    grouped = data.groupby(['date', group_col])['num_sold'].sum().reset_index()
    
    # Calculate global totals by date
    global_totals = data.groupby('date')['num_sold'].sum().reset_index()
    global_totals.rename(columns={'num_sold': 'num_sold_global'}, inplace=True)
    
    # Merge grouped data with global totals
    merged = grouped.merge(global_totals, on='date')
    merged['fractions'] = merged['num_sold'] / merged['num_sold_global']
    
    unique_groups = np.sort(merged[group_col].unique())
    colors = cm.get_cmap(colormap, len(unique_groups))
    
    for i, group in enumerate(unique_groups):
        mask = merged[group_col] == group
        ax.plot(
            merged[mask]['date'], 
            merged[mask]['fractions'], 
            label=group, 
            color=colors(i)  
        )
    
    ax.legend(bbox_to_anchor=(1, 1), title=group_col.capitalize())
    ax.set_xlabel('Date')
    ax.set_ylabel('Fraction of Total Sales')
    ax.grid(True, linestyle='--', alpha=0.6)


In [ ]:
if __name__ == "__main__":
    # Display country weights
    print("Country Weights:")
    print(country_weights)

    # Decompose and plot country fractions
    fig, ax = plt.subplots(figsize=(10, 6))
    decompose(train_df_imputed, 'country', ax, colormap='plasma')
    ax.set_title("Country Fractions Over Time")
    plt.show()

    # Display store weights
    print("Store Weights:")
    print(store_weights)

    # Decompose and plot store fractions
    fig, ax = plt.subplots(figsize=(10, 6))
    decompose(train_df_imputed, 'store', ax, colormap='plasma')
    ax.set_title("Store Fractions Over Time")
    plt.show()

    # Display product weights
    print("Product Weights:")
    print(product_weights)

    # Decompose and plot product fractions
    fig, ax = plt.subplots(figsize=(10, 6))
    decompose(train_df_imputed, 'product', ax, colormap='plasma')
    ax.set_title("Product Fractions Over Time")
    plt.show()


## Product Ratios and Forecasting
Calculating normalized sales ratios for each product and forecasting future trends based on past data.

In [ ]:
def forecast_product_ratios(train_df, forecast_years):
    """
    Forecast product ratios for specific years based on historical data.

    """
    # Calculate product fractions by day
    product_df = train_df.groupby(["date", "product"])["num_sold"].sum().reset_index()
    
    # Pivot to get a DataFrame where each column represents a product
    product_ratio_df = product_df.pivot(index="date", columns="product", values="num_sold")
    
    # Normalize each row to calculate product ratios
    product_ratio_df = product_ratio_df.div(product_ratio_df.sum(axis=1), axis=0)
    product_ratio_df = product_ratio_df.stack().rename("ratios").reset_index()
    
    # Forecast product ratios for the given years
    forecasted_ratios = []
    for base_year, target_year, year_shift in forecast_years:
        # Filter data for the base year
        forecast_df = product_ratio_df[product_ratio_df["date"].dt.year == base_year].copy()
        # Shift the date to the target year
        forecast_df["date"] += pd.DateOffset(years=year_shift)
        forecasted_ratios.append(forecast_df)
    
    forecasted_ratios_df = pd.concat(forecasted_ratios, ignore_index=True)
    
    return forecasted_ratios_df


* **Date Offsets:**

    * For 2017, offset by 2 years from 2015.
    * For 2018, offset by 2 years from 2016.
    * For 2019, offset by 4 years from 2015.

* **Output**
The `forecasted_ratios_df` will contain the product ratios for:

    * 2017 (based on 2015 data).
    * 2018 (based on 2016 data).
    * 2019 (based on 2015 data).

In [ ]:
# Define the input DataFrame (train_df_imputed) and forecast years
forecast_years = [(2015, 2017, 2), (2016, 2018, 2), (2015, 2019, 4)]

forecasted_ratios_df = forecast_product_ratios(train_df_imputed, forecast_years)

# Display a sample of the forecasted ratios
print("Forecasted Product Ratios (Sample):")
print(forecasted_ratios_df.head(5))

## Time Series Aggregation for Total Sales
Aggregating sales data by time periods (date, year, month, day of the week) for feature extraction.

In [ ]:
# Create a copy of the DataFrame
original_train_df_imputed = train_df_imputed.copy()

# Aggregate total sales by date
train_df_imputed = train_df_imputed.groupby(["date"])["num_sold"].sum().reset_index()

# Extract year, month, day, and day of the week from the date
train_df_imputed["year"] = train_df_imputed["date"].dt.year
train_df_imputed["month"] = train_df_imputed["date"].dt.month
train_df_imputed["day"] = train_df_imputed["date"].dt.day
train_df_imputed["day_of_week"] = train_df_imputed["date"].dt.dayofweek


## Feature Correlation Analysis

In [ ]:
correlation_features = [
    'year', 
    'month', 
    'day', 
    'day_of_week', 
    'num_sold'
]

# Compute the correlation matrix
correlation_matrix = train_df_imputed[correlation_features].corr()

plt.figure(figsize=(7, 5))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="plasma", cbar=True, square=True, linewidths=0.5)
plt.title("Correlation Heatmap of Features", fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()

The heatmap visualizes the correlation between different features in the dataset: `year`, `month`, `day`, `day_of_week`, and `num_sold`.

**Specific Insights:**

* **`num_sold` vs. `day_of_week`:** There is a moderate positive correlation (0.58) between the number of items sold (`num_sold`) and the day of the week. This suggests that sales activity might be influenced by the day of the week.

* **`num_sold` vs. `year`:** A weak negative correlation (-0.27) exists between `num_sold` and `year`. This could indicate a slight decrease in sales over the years represented in the dataset. However, the correlation is weak, so further investigation is needed to confirm this trend.

* **Other correlations:**
    * `month` and `day` have a very weak positive correlation (0.01).
    * `day_of_week` has a moderate positive correlation (0.58) with itself.
    * All other correlations are very weak or negligible.

**Possible Implications:**

* **Day-of-the-week effect:** The moderate positive correlation between `num_sold` and `day_of_week` suggests that sales might be higher on certain days of the week. Further analysis could reveal which days have the highest and lowest sales.
* **Year-over-year trend:** The weak negative correlation between `num_sold` and `year` might indicate a slight decline in sales over time. However, more data and analysis are needed to confirm this trend and understand the underlying reasons.

## Monthly Sales Trends
Aggregating sales by year and month and visualizing monthly trends.

In [ ]:
# Aggregate total sales by year and month
monthly_sales = train_df_imputed.groupby(['year', 'month'])['num_sold'].sum().reset_index()

# Create a 'date' column for monthly aggregation
monthly_sales['date'] = pd.to_datetime(monthly_sales[['year', 'month']].assign(day=1))


In [ ]:
# Set up the line plot for monthly sales trends
plt.figure(figsize=(14, 7))
sns.lineplot(
    data=monthly_sales,
    x='date',
    y='num_sold',
    marker='o',
    color=sns.color_palette("plasma", as_cmap=True)(0.5) 
)

plt.title("Monthly Sales Trends", fontsize=14)
plt.xlabel("Month", fontsize=12)
plt.ylabel("Total Sales", fontsize=12)
plt.grid(True, linestyle="--", alpha=0.6)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


The plot visualizes the trend of monthly total sales over a period from 2010 to 2017.
The sales figures appear to fluctuate significantly over time.

**Specific Insights:**

* **Volatility:** The plot shows a high degree of volatility in monthly sales. There are periods of rapid increases followed by sharp declines, indicating that sales are not stable.
* **No clear trend:** It's difficult to identify a consistent upward or downward trend in sales over the entire period. The fluctuations seem to be random rather than following a clear pattern.
* **Potential Seasonality:** While not immediately apparent, there might be some subtle seasonal patterns within the volatility. For example, there could be slight increases or decreases in sales during certain months or quarters.

## Sales Trends Heatmap
Creating a heatmap to show sales distribution across years and months.

In [ ]:
# Pivot the data for heatmap
heatmap_data = monthly_sales.pivot(index="year", columns="month", values="num_sold")

plt.figure(figsize=(12, 8))
sns.heatmap(
    heatmap_data,
    annot=True, fmt=".0f", cmap="plasma", linewidths=0.5, cbar=True
)

plt.title("Sales Trends by Month and Year", fontsize=14)
plt.xlabel("Month", fontsize=12)
plt.ylabel("Year", fontsize=12)
plt.xticks(ticks=range(1, 13), labels=[
    "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", 
    "Aug", "Sep", "Oct", "Nov", "Dec"
], rotation=45, fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()


The heatmap visualizes monthly sales trends from 2010 to 2016.

**Specific Insights:**

* **Year-over-Year Trends:** 
    * There is a general upward trend in sales from 2010 to 2016. 
    * However, the growth rate seems to vary across years. Some years show more significant increases compared to others.

* **Monthly Seasonality:** 
    * Sales appear to be higher in certain months throughout the years. 
    * December consistently shows the highest sales volumes, suggesting a strong year-end seasonality.
    * Other months with relatively high sales include November and October, possibly indicating pre-holiday shopping trends.

* **Month-to-Month Variation:** 
    * Within each year, there is substantial variation in sales across different months. 
    * This suggests that factors beyond just year-over-year trends are influencing sales performance.

**Possible Implications:**

* **Seasonality Management:** The strong December sales suggest that businesses should prepare for increased demand and inventory management during this period.
* **Marketing Strategies:** Targeted marketing campaigns during peak sales months (December, November, October) could help capitalize on the increased demand.
* **Inventory Planning:** Understanding the seasonal patterns can help businesses optimize inventory levels throughout the year, avoiding stockouts during peak demand periods and minimizing excess inventory during slower months.

## Preparing Test Data for Forecasting
Adding new columns like month, day, and day_of_week to facilitate downstream analysis and forecasting.

In [ ]:
test_total_sales_df = test_df.groupby(["date"])["id"].first().reset_index().drop(columns="id")
test_total_sales_df["month"] = test_total_sales_df["date"].dt.month
test_total_sales_df["day"] = test_total_sales_df["date"].dt.day
test_total_sales_df["day_of_week"] = test_total_sales_df["date"].dt.dayofweek


## Calculating Weekly Sales Ratios
Calculates the daily sales ratios as a percentage of the weekly total sales. It also aggregates the weekly sales data and adds helpful columns for week identification and day-wise ratios.

In [ ]:
df = train_df_imputed.copy()
df['iso_year'] = df['date'].dt.isocalendar().year 
df['iso_week'] = df['date'].dt.isocalendar().week 
df['week_id'] = df['iso_year'].astype(str) + '-W' + df['iso_week'].astype(str).str.zfill(2)
df['day_of_week'] = df['date'].dt.dayofweek

# Aggregate weekly total sales
weekly_total = df.groupby('week_id')['num_sold'].sum().reset_index()
weekly_total.rename(columns={'num_sold': 'weekly_total_sold'}, inplace=True)

# Merge weekly totals back to the original DataFrame
df = pd.merge(df, weekly_total, on='week_id')

# Calculate daily sales ratio as a percentage of weekly total sales
df['daily_sales_ratio'] = df['num_sold'] / df['weekly_total_sold']

# Aggregate weekly ratios by day of the week
weekly_ratio = df.groupby(['week_id', 'day_of_week'])['daily_sales_ratio'].sum().reset_index()

# Get the start date for each week (Monday)
weekly_ratio['week_start'] = pd.to_datetime(weekly_ratio['week_id'] + '-1', format='%Y-W%W-%w')
first_monday = weekly_ratio['week_start'].min()

# Define days of the week for labeling in plots
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']


## Visualization of Weekly Sales Ratio by Day of Week
Observe the fluctuation of daily sales ratios over weeks. Each day of the week is plotted with a distinct color for better interpretation.

In [ ]:
palette = sns.color_palette("plasma_r", 7)  

plt.figure(figsize=(14, 7))
for day in range(7):
    day_data = weekly_ratio[weekly_ratio['day_of_week'] == day]
    plt.plot(
        day_data['week_start'], 
        day_data['daily_sales_ratio'], 
        label=days[day], 
        marker='o', 
        color=palette[day]  
    )

plt.xlim(left=first_monday)
plt.ylim(top=0.3)
plt.xlabel('Week Start Date (Monday)', fontsize=12)
plt.ylabel('Daily Sales Ratio (Percentage of Weekly Total)', fontsize=12)
plt.title('Weekly Sales Ratio by Day of Week', fontsize=14)
plt.legend(title='Day of Week', fontsize=10)
plt.grid(True, linestyle='--', alpha=0.6)  
plt.xticks(rotation=45, fontsize=10)
plt.tight_layout()
plt.show()


The plot visualizes the daily sales ratio (percentage of weekly total sales) for each day of the week over a period from 2010 to 2017.
Each line represents a different day of the week (Monday to Sunday).

**Specific Insights:**

* **Weekday vs. Weekend Sales:** There is a clear distinction between weekday and weekend sales patterns.
    * Weekdays (Monday to Friday) generally have lower sales ratios compared to weekends (Saturday and Sunday).
    * This suggests that a significant portion of sales activity occurs during the weekend.
      
* **Day-to-Day Variations:** Even within weekdays, there are variations in sales ratios.
    * Some weekdays consistently have lower sales ratios than others. For example,  Monday,Thursday, Wednesday and Thursday might have slightly lower sales compared to Friday.
    * However, these variations are generally less pronounced compared to the difference between weekdays and weekends.

* **Fluctuations Over Time:** There are some fluctuations in the sales ratios over the years.


## Total Sales by Day of the Week
Aggregates total sales by day of the week and showcases the sales distribution for each day.

In [ ]:
# Aggregate sales by day of the week
day_of_week_sales = train_df_imputed.groupby("day_of_week")["num_sold"].sum()
day_of_week_sales.index = days  

plt.figure(figsize=(10, 5))
sns.barplot(x=day_of_week_sales.index, y=day_of_week_sales.values, palette="plasma_r")
plt.title("Total Sales by Day of the Week", fontsize=14)
plt.xlabel("Day of the Week", fontsize=12)
plt.ylabel("Total Sales", fontsize=12)
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()


The plot visualizes the total sales volume for each day of the week.
There is a clear upward trend in sales from Monday to Sunday.

**Specific Insights:**

* **Sunday Sales Dominance:** Sunday has the highest total sales among all days of the week.
* **Weekday Sales Patterns:** Weekday sales generally increase from Monday to Friday, with a noticeable jump in sales from Friday to Saturday.
* **Weekend Peak:** The significant increase in sales from Friday to Saturday and then to Sunday indicates a strong weekend sales trend.

**Possible Implications:**

* **Customer Behavior:** This pattern suggests that a significant portion of the customer base prefers to shop on weekends. This could be due to factors like leisure time, family time, or weekend promotions.
* **Staffing and Scheduling:** Businesses can optimize staffing levels and scheduling based on expected sales volumes on different days of the week. More staff may be needed on weekends to handle the increased customer traffic and sales.
* **Marketing and Promotions:** Targeted marketing campaigns or promotions could be implemented on weekdays to encourage more sales during the week.
* **Inventory Management:** Understanding the weekend-focused sales can help businesses manage inventory levels to ensure sufficient stock availability on weekends and minimize excess inventory on weekdays.

## Sales Distribution Across Days and Months (Heatmap)

In [ ]:
# Group by month and day of the week, summing the sales
monthly_day_sales = df.groupby(['month', 'day_of_week'])['num_sold'].sum().reset_index()

# Define days of the week for labeling in plots
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Pivot the data for heatmap: rows = days, columns = months, values = sales
heatmap_data = monthly_day_sales.pivot(index='day_of_week', columns='month', values='num_sold')

# Set day names for rows; ensure index aligns with expected day order
heatmap_data.index = days[:len(heatmap_data)]  

plt.figure(figsize=(12, 8))
sns.heatmap(
    heatmap_data, 
    annot=True, fmt=".0f", cmap="plasma", linewidths=0.5, cbar=True
)

plt.title("Aggregate Sales by Month and Day of Week", fontsize=14)
plt.xlabel("Month", fontsize=12)
plt.ylabel("Day of the Week", fontsize=12)
plt.xticks(ticks=range(1, 13), labels=["Jan", "Feb", "Mar", "Apr", "May", "Jun", 
                                          "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"], rotation=45, fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()


The heatmap visualizes the total sales volume for each combination of month and day of the week.
Color intensity represents the total sales volume.

**Specific Insights:**

* **Day of the Week:**
    * **Sunday** consistently shows the highest sales volumes across all months.
    * Sales generally increase from Monday to Sunday, with a significant jump from Friday to Saturday.
    * Weekday sales (Monday to Friday) are relatively lower compared to weekends.

* **Monthly Trends:**
    * December consistently shows the highest sales across all days of the week.
    * November and October also tend to have high sales volumes.
    * January typically shows lower sales compared to other months.

* **Combined Trends:**
    * The combination of Sunday and December shows the highest sales volume.
    * Weekday sales are generally lower, with Monday and Tuesday showing the lowest sales volumes.


## Calculating Day of Week Ratios and Adjusting Sales

In [ ]:
# Calculate Day of Week Ratios and Adjust Sales

# Calculate the average sales for each day of the week and normalize it
day_of_week_ratio = (
    train_df_imputed.groupby("day_of_week")["num_sold"].mean() / 
    train_df_imputed.groupby("day_of_week")["num_sold"].mean().mean()
).rename("day_of_week_ratios")

# Display day of week ratios
display(day_of_week_ratio)

# Merge day of week ratios back into the main DataFrame
train_df_imputed = pd.merge(train_df_imputed, day_of_week_ratio, how="left", on="day_of_week")

# Adjust num_sold based on day of week ratios
train_df_imputed["adjusted_num_sold"] = train_df_imputed["num_sold"] / train_df_imputed["day_of_week_ratios"]

# Check the difference for Adjusted Sales
difference_check = (train_df_imputed["num_sold"].sum() - train_df_imputed["adjusted_num_sold"].sum()) / train_df_imputed["num_sold"].sum()
print(f"The difference between original and adjusted total sales as a proportion is: {difference_check:.6f}")

# Print adjusted_num_sold values
print("\nAdjusted Sales (Adjusted num_sold):")
print(train_df_imputed[["date", "num_sold", "adjusted_num_sold"]].head())  


**Day-of-Week Effect:**

* **Weekdays (Monday-Thursday):** Sales on these days are generally lower than the average, with ratios consistently around 0.94. This suggests that sales activity is subdued during the weekdays.
* **Friday:** Sales on Friday are close to the average, with a ratio near 1. This indicates that sales on Fridays are relatively typical compared to the overall average sales across all days.
* **Saturday:** Sales on Saturday show an increase compared to the average, with a ratio of 1.056. This suggests that sales activity picks up on Saturdays.
* **Sunday:** Sales on Sunday are significantly higher than the average, with the highest ratio of 1.169. This indicates that Sundays are the busiest sales days.

**These ratios demonstrate a clear weekend sales trend, with sales increasing from weekdays to Saturday and peaking on Sunday.** 

* **Minimal Impact of Adjustment:** The very small difference between the original and adjusted total sales (0.000022) indicates that the adjustment based on day-of-week ratios has a negligible impact on the overall sales volume. This suggests that the average sales across different days of the week are relatively balanced.

## Making the Forecast
Prepare test data for forecasting by merging the adjusted daily sales averages and day-of-week ratios into the test dataset. This enables accurate forecast generation.


In [ ]:
# Function to Prepare Test Data for Forecasting
def prepare_test_data(train_df_imputed, test_total_sales_df, day_of_week_ratio):
    """
    Prepare the test data by calculating daily mean sales and incorporating day-of-week ratios.
    
    """
    # Filter training data for the last X years (from 2010 onwards)
    train_last_x_years_df = train_df_imputed.loc[train_df_imputed["year"] >= 2010]
    
    # Calculate daily mean of adjusted sales for each month and day
    train_day_mean_df = train_last_x_years_df.groupby(["month", "day"])["adjusted_num_sold"].mean().reset_index()
    
    # Merge average daily sales into the test DataFrame
    test_total_sales_df = pd.merge(test_total_sales_df, train_day_mean_df, how="left", on=["month", "day"])
    
    # Merge with day-of-week ratios
    test_total_sales_df = pd.merge(test_total_sales_df, day_of_week_ratio.reset_index(), how="left", on="day_of_week")
    
    # Calculate forecasted daily sales
    test_total_sales_df["num_sold"] = test_total_sales_df["adjusted_num_sold"] * test_total_sales_df["day_of_week_ratios"]
    
    return test_total_sales_df


## Disaggregating Total Sales Forecast
Disaggregates the total sales forecast by combining store, country, and product-level ratios with the daily sales forecast, producing fine-grained sales predictions.


In [ ]:
# Function to Disaggregate Total Sales Forecast
def disaggregate_forecast(test_df, test_total_sales_df, store_weights, gdp_per_capita_filtered_ratios_df, forecasted_ratios_df):
    """
    Disaggregate total sales forecast by incorporating store, country, and product ratios.
    
    """
    # Add store ratios
    store_weights_df = store_weights.reset_index()
    test_sub_df = pd.merge(test_df, test_total_sales_df, how="left", on="date")
    test_sub_df.rename(columns={"num_sold": "day_num_sold"}, inplace=True)
    
    # Add product ratios
    test_sub_df = pd.merge(test_sub_df, store_weights_df, how="left", on="store")
    test_sub_df.rename(columns={"num_sold": "store_ratio"}, inplace=True)
    
    # Add country ratios
    test_sub_df["year"] = test_sub_df["date"].dt.year
    test_sub_df = pd.merge(test_sub_df, gdp_per_capita_filtered_ratios_df, how="left", on=["year", "country"])
    test_sub_df.rename(columns={"ratio": "country_ratio"}, inplace=True)
    
    # Add product ratios
    test_sub_df = pd.merge(test_sub_df, forecasted_ratios_df, how="left", on=["date", "product"])
    test_sub_df.rename(columns={"ratios": "product_ratio"}, inplace=True)
    
    # Adjust for bias for Kenya's GDP ratio
    test_sub_df.loc[test_sub_df['country'] == 'Kenya', 'country_ratio'] += 0.00249144564 * 1 / 10
    
    # Calculate final forecasted `num_sold`
    test_sub_df["num_sold"] = (
        test_sub_df["day_num_sold"] * 
        test_sub_df["store_ratio"] * 
        test_sub_df["country_ratio"] * 
        test_sub_df["product_ratio"]
    )
    
    # Round `num_sold` to nearest integer
    test_sub_df["num_sold"] = test_sub_df["num_sold"].round()
    
    return test_sub_df


## Plotting Individual Time Series
Generates time series plots for individual categories, enabling a side-by-side comparison of actual and forecasted trends.

In [ ]:
# Function to Plot Individual Time Series
def plot_individual_ts(df):
    """
    Plot individual time series for each combination of country, store, and product.
    
    """
    # Generate a color palette using Plasma
    unique_countries = df["country"].unique()
    colour_map = sns.color_palette("plasma", len(unique_countries))
    
    country_color_map = {country: colour_map[i] for i, country in enumerate(unique_countries)}
    
    for country in unique_countries:
        f, axes = plt.subplots(df["store"].nunique() * df["product"].nunique(), figsize=(20, 70))
        count = 0
        
        for store in df["store"].unique():
            for product in df["product"].unique():
                plot_data = df.loc[
                    (df["product"] == product) & 
                    (df["country"] == country) & 
                    (df["store"] == store)
                ]
                sns.lineplot(data=plot_data, x="date", y="num_sold", linewidth=0.5,
                             ax=axes[count], color=country_color_map[country])
                axes[count].set_title(f"{country} - {store} - {product}")
                axes[count].axvline(pd.to_datetime("2017-01-01"), color='black', linestyle='--')
                axes[count].grid(True, linestyle='--', alpha=0.6)
                count += 1


In [ ]:
# Prepare the forecasted data
test_total_sales_forecasted = prepare_test_data(train_df_imputed, test_total_sales_df, day_of_week_ratio)

# Disaggregate the forecasted data
test_forecast_disaggregated = disaggregate_forecast(
    test_df,
    test_total_sales_forecasted,
    store_weights,
    gdp_per_capita_filtered_ratios_df,
    forecasted_ratios_df
)

# Plot individual time series using both original and forecasted data
plot_individual_ts(pd.concat([original_train_df_imputed, test_forecast_disaggregated]).reset_index(drop=True))


## Creating the Submission File

In [ ]:
# Create submission file
sample_df = pd.read_csv("/kaggle/input/playground-series-s5e1/sample_submission.csv")
sample_df["num_sold"] = test_forecast_disaggregated["num_sold"]

display(sample_df.head(5))

sample_df.to_csv("submission.csv", index=False)
print("Submission file created.")

<!-- Include Google Fonts for a modern font -->
<link href="https://fonts.googleapis.com/css2?family=Roboto:wght@700&display=swap" rel="stylesheet">

<div style="border-radius: 15px; border: 2px solid #8455a1; padding: 20px; 
           background: linear-gradient(135deg, #fcaed5, #fabba2); 
           text-align: left; box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.5);">
    <div style="background-color: rgba(255, 255, 255, 0.2); padding: 20px; border-radius: 10px;">
        <h3 style="color: #6A1B9A; font-weight: bold; margin-bottom: 10px; font-size: 18px; font-family: 'Roboto', sans-serif; line-height: 1.2;">
            References :
        </h3>
        <p style="color: #8455a1; font-size: 16px; margin-bottom: 10px;">
            [S5E1] Previous Years Baseline - No Model: 
            <a href="https://www.kaggle.com/code/cabaxiom/s5e1-previous-years-baseline-no-model" style="color: #8455a1; text-decoration: underline;">https://www.kaggle.com/code/cabaxiom/s5e1-previous-years-baseline-no-model</a>
        </p>
        <p style="color: #8455a1; font-size: 16px; margin-bottom: 10px;">
            [S5E1] EDA and Linear Regression Baseline: 
            <a href="https://www.kaggle.com/code/cabaxiom/s5e1-eda-and-linear-regression-baseline?scriptVersionId=216127469" style="color: #8455a1; text-decoration: underline;">https://www.kaggle.com/code/cabaxiom/s5e1-eda-and-linear-regression-baseline</a>
        </p>
        <p style="color: #8455a1; font-size: 16px; margin-bottom: 10px;">
            EDA and using previous ratio as prediction: 
            <a href="https://www.kaggle.com/code/act18l/eda-and-using-previous-ratio-as-prediction" style="color: #8455a1; text-decoration: underline;">https://www.kaggle.com/code/act18l/eda-and-using-previous-ratio-as-prediction</a>
        </p>
        <p style="color: #8455a1; font-size: 16px;">
            Discussion Forums:
            <a href="https://www.kaggle.com/competitions/playground-series-s5e1/discussion/554349" style="color: #8455a1; text-decoration: underline;">https://www.kaggle.com/competitions/playground-series-s5e1/discussion/554349</a>,
            <a href="https://www.kaggle.com/competitions/playground-series-s5e1/discussion/555500" style="color: #8455a1; text-decoration: underline;">https://www.kaggle.com/competitions/playground-series-s5e1/discussion/555500</a>
        </p>
        <p style="color: #8455a1; font-size: 16px;">
            Competition: 
            <a href="https://kaggle.com/competitions/playground-series-s5e1" style="color: #8455a1; text-decoration: underline;">Playground Series S5E1: Forecasting Sticker Sales</a> by Walter Reade and Elizabeth Park.
        </p>
    </div>
</div>


<!-- Include Google Fonts for a modern font -->
<link href="https://fonts.googleapis.com/css2?family=Roboto:wght@700&display=swap" rel="stylesheet">

<div style="border-radius: 15px; border: 2px solid #6A1B9A; padding: 20px; 
           background: linear-gradient(135deg, #6610f2, #ff99cc, #ffa07a, #ffff00); 
           text-align: center; box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.5);">
    <h1 style="color: #ffffff; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.7); 
               font-weight: bold; margin-bottom: 10px; font-size: 28px; 
               font-family: 'Roboto', sans-serif; line-height: 1.2;">
        🙏 Thanks for Reading! 🚀
    </h1>
    <p style="color: #ffffff; font-size: 22px; text-align: center;">
        Happy Coding! 🙌😊
    </p>
</div>
